<a href="https://colab.research.google.com/github/dcugini/ML_course_Pavia_23/blob/main/Schwinger_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SCHWINGER MODEL ON THE CHAIN
The Schwinger model is a Quantum Field Theory an represents a Toy Model of the Standard Model.
The study of this Model is of interest in the context of Quantum Siumlations,
since it is 1-dimensional in the space coordinate.
Indeed, after having discretized the space (via the Kougut-Susskind approach) ,we are left with a lattice were each can be eventually occupated by a fermion or an antifermion.
In particular, even sites can be filled by fermions,
odd sites by antifermions.
On the other hand the link between two consecutive sites represents the Electric field between them.


In [19]:
import numpy as np
import random

In [90]:
N = 3
Delta = 1

n_chains_train = 10
n_chains_test = 0





In [91]:
n_qubits = 2*N

## Data generation

In [92]:
random.seed(121)

x = []

for i in range(0,n_chains_train + n_chains_test):
  x.append(random.randrange(2**n_qubits))



In [97]:
def f (x):
  PBC = False
  E = 0
  min = 0
  max = 0
  for i in range(0, n_qubits):
    E += ((-1)**i)*(x&(2**i))/(2**i)
    if E < min:
      min -= 1
    if E > max:
      max += 1

  if E == 0:
    PBC = True
  return PBC, (max-min)

def label (x):
  y = f(x)
  if (y[0] == True) and (y[1] <= Delta):
    r = 1
  else:
    r = 0
  return r



In [98]:
for i in range(0,10):
  print('{:0{}b}'.format(x[i], n_qubits),f(x[i]))

001011 (False, 2)
011011 (True, 2)
110010 (False, 1)
010110 (False, 2)
010110 (False, 2)
101010 (False, 3)
100100 (True, 1)
010111 (False, 2)
110110 (True, 2)
000111 (False, 1)


In [99]:
for i in range(0,10):
  print('{:0{}b}'.format(x[i], n_qubits),label(x[i]))

001011 0
011011 0
110010 0
010110 0
010110 0
101010 0
100100 1
010111 0
110110 0
000111 0
